# Deploy a model to a secure AKS

# 1. Connect to Azure Machine Learning Workspace

## 1.1. Import the required libraries

In [ ]:
%pip install azure.ai.ml --extra-index-url  https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2

In [22]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model, Environment, KubernetesOnlineEndpoint, KubernetesOnlineDeployment, CodeConfiguration
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities._deployment.resource_requirements_settings import ResourceRequirementsSettings
from azure.ai.ml.entities._deployment.container_resource_settings import ResourceSettings

## 1.2. Configure workspace details and get a handle to the workspace

In [4]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [8]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Create a model

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model = Model(
    path="model/model.pkl",
    type=AssetTypes.CUSTOM_MODEL,
    name="diabetes_model_example",
)
ml_client.models.create_or_update(model)

# 3. Create an inference environment for the model

## Environment

[Note about no-code-deployment](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?tabs=cli) : Azure Machine Learning performs dynamic installation of packages when deploying MLflow models with no-code deployment. As a consequence, deploying MLflow models to online endpoints with no-code deployment in a private network without egress connectivity is not supported by the moment.

This means we have to provide an environment and a scoring files for secured infrastructure.

All curated environment are part of mcr.microsoft.com and most customers do not want to allow public network trafic from/to their inference endpoint.

This means you have to create a custom environment that will be saved in your private container registry. You have 2 ways to do this :
- Define a custom environment within AML and let the image-builder cluster build it and save it in ACR for you. The cluster will require access to the source you are using.
- Use your own platform (a side VM ?) to create and push your docker image to the ACR, and then register it to AML.

### Build a custom environment within AML

Network prerequisite : image-builder cluster (within training subnet) can to reach out to mcr.microsoft.com to retrieve curated image as a base image to build the custom one that will be saved into private ACR. (This is already implemented as part as the terraform code).

The following command will :
- Define the environment 
- Register the environment
- Build it on imabe_builder cluster (because we defined this cluster as default during installation) 
- Save the environment in Azure contrainer registry

In [ ]:
env = Environment(
    image="mcr.microsoft.com/azureml/curated/minimal-ubuntu18.04-py37-cpu-inference:51",
    conda_file="environment/conda.yaml",
    name="my_custom_inference_env",
)
ml_client.environments.create_or_update(env)

# Setup a Kubernetes online endpoint

This takes around 2mins.

In [ ]:
import datetime

online_endpoint_name = "k8s-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = KubernetesOnlineEndpoint(
    name=online_endpoint_name,
    compute="<AKS_COMPUTE_NAME>",
    auth_mode="key",
)

ml_client.begin_create_or_update(endpoint).result()

# Create the online deployment



In [ ]:
deployment = KubernetesOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="src", scoring_script="score.py"
    ),
    instance_count=1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="100m",
            memory="0.5Gi",
        ),
    )      
)